In [1]:
# Install Ultralytics (YOLO) if not already installed
%pip install ultralytics
import ultralytics
ultralytics.checks()

import os
import shutil
from ultralytics import YOLO

# --- PATH CONFIGURATION (Your original paths) ---
BASE = "/kaggle/working"
DATASET = "/kaggle/working/space-dataset"

# Make sure these input paths exist in your Kaggle environment
TRAIN_SRC = "/kaggle/input/duality-ai/hackathon2_train_2/train_2"
TEST_SRC = "/kaggle/input/duality-ai/Hackathon2_test1/test1"

# --- DATASET SETUP ---
if os.path.exists(DATASET):
    shutil.rmtree(DATASET)
os.makedirs(DATASET, exist_ok=True)

print("📂 TRAIN source:", TRAIN_SRC)
print("📂 TEST source (external):", TEST_SRC)

print("\n📁 Creating YOLO train/val structure...")

# Copying train and val data to the working directory
shutil.copytree(f"{TRAIN_SRC}/train2", f"{DATASET}/train")
shutil.copytree(f"{TRAIN_SRC}/val2", f"{DATASET}/val")

print("✅ TRAIN and VAL copied successfully!")

print("\n⚠️ TEST will NOT be copied (to avoid disk full).")
print(" YOLO will read test images directly from:", f"{TEST_SRC}/images")

classes = [
    "OxygenTank",
    "NitrogenTank",
    "FirstAidBox",
    "FireAlarm",
    "SafetySwitchPanel",
    "EmergencyPhone",
    "FireExtinguisher"
]

# Create classes.txt (Optional for training, but good for reference)
with open(f"{DATASET}/classes.txt", "w") as f:
    f.write("\n".join(classes))
print("✔ classes.txt created")

# --- YAML CREATION ---
yaml_path = f"{DATASET}/yolo_params.yaml"
yaml_content = f"""
path: {DATASET}
train: train/images
val: val/images

# test set is external:
test: {TEST_SRC}/images

nc: {len(classes)}
names: {classes}
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f"✔ yolo_params.yaml created at: {yaml_path}")

# --- YOLO V11 M TRAINING ---
print("\n🚀 Starting YOLO11m Training...")

# Load the YOLO v11 Medium model
# Ultralytics will automatically download 'yolo11m.pt' if not present


# Train the model

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6582.3/8062.4 GB disk)
📂 TRAIN source: /kaggle/input/duality-ai/hackathon2_train_2/train_2
📂 TEST source (external): /kaggle/input/duality-ai/Hackathon2_test1/test1

📁 Creating YOLO train/val structure...
✅ TRAIN and VAL copied successfully!

⚠️ TEST will NOT be copied (to avoid disk full).
 YOLO will read test images directly from: /kaggle/input/duality-ai/Hackathon2_test1/test1/images
✔ classes.txt created
✔ yolo_params.yaml created at: /kaggle/working/space-dataset/yolo_params.yaml

🚀 Starting YOLO11m Training...


In [2]:

model = YOLO("yolo12m.pt") 


In [8]:

results = model.train(
    data=yaml_path,     # Path to the dataset config we just created
    epochs=100,          # Adjust epochs as needed
    imgsz=720,          # Image size
    batch=8,
    device=[0,1],
   # Adjust batch size based on GPU memory (16 is usually safe for 'm' on T4)
    project="/kaggle/working/runs/train", # Explicit path to save results
    name="yolo11m_space_run",             # Name of this specific run
    exist_ok=True,
    patience = 10 # Overwrite existing experiment with same name
)

print("🏆 Training Complete. Best model saved in /kaggle/working/runs/train/yolo11m_space_run/weights/")

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/space-dataset/yolo_params.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=720, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11m_space_run, nbs=64, nms=False, o

In [9]:
print("\n📊 Validating model performance...")
metrics = model.val() 

# Print key metrics explicitly
print("-" * 30)
print(f"✔ mAP@50 (Metric for detection):    {metrics.box.map50:.4f}")
print(f"✔ mAP@50-95 (Strict accuracy):      {metrics.box.map:.4f}")
print(f"✔ Precision:                        {metrics.box.mp:.4f}")
print(f"✔ Recall:                           {metrics.box.mr:.4f}")
print("-" * 30)

# --- 3. PREDICT ON EXTERNAL TEST SET ---
# Since you kept the test set in the input folder (to save space), we point to it here.
print(f"\n🔮 Running inference on Test Set: {TEST_SRC}/images")

test_results = model.predict(
    source=f"{TEST_SRC}/images",
    save=True,              # Save the images with bounding boxes drawn
    save_txt=True,          # Save the detection coordinates (.txt)
    conf=0.25,              # Minimum confidence to detect an object
    project="space_mission",
    name="test_predictions" ,
    # Results will be in: space_mission/test_predictions
)

print(f"✅ Testing complete! Results saved to: {BASE}/space_mission/test_predictions")


📊 Validating model performance...
Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12m summary (fused): 169 layers, 20,110,309 parameters, 0 gradients, 67.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4906.3±832.4 MB/s, size: 2848.2 KB)
val: Scanning /kaggle/working/space-dataset/val/labels.cache... 336 images, 16 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 336/336 61.7Kit/s 0.0ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 21/21 1.7it/s 12.3s0.6s
                   all        336        957      0.934      0.818      0.875      0.771
            OxygenTank        135        237      0.964      0.865      0.922      0.853
          NitrogenTank        157        283      0.918      0.845      0.898      0.809
           FirstAidBox         97        119      0.902      0.849      0.901      0.818
             FireAlarm         55         57      0.924      0.854      0.914

In [13]:
! nvidia-smi

Fri Dec 19 17:30:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             32W /  250W |   16267MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# EPOCHS = 10
# MOSAIC = 0.4
# OPTIMIZER = 'AdamW'
# MOMENTUM = 0.9
# LR0 = 0.0001
# LRF = 0.0001
# SINGLE_CLS = False
# import argparse
# from ultralytics import YOLO
# import os
# import sys

# if __name__ == '__main__': 
#     parser = argparse.ArgumentParser()
#     # epochs
#     parser.add_argument('--epochs', type=int, default=EPOCHS, help='Number of epochs')
#     # mosaic
#     parser.add_argument('--mosaic', type=float, default=MOSAIC, help='Mosaic augmentation')
#     # optimizer
#     parser.add_argument('--optimizer', type=str, default=OPTIMIZER, help='Optimizer')
#     # momentum
#     parser.add_argument('--momentum', type=float, default=MOMENTUM, help='Momentum')
#     # lr0
#     parser.add_argument('--lr0', type=float, default=LR0, help='Initial learning rate')
#     # lrf
#     parser.add_argument('--lrf', type=float, default=LRF, help='Final learning rate')
#     # single_cls
#     parser.add_argument('--single_cls', type=bool, default=SINGLE_CLS, help='Single class training')
#     args, unknown = parser.parse_known_args()
    
   
#     model = YOLO("yolov8s.pt")
#     results = model.train(
#         data=yaml_path, 
#         epochs=args.epochs,
#         device=0,
#         single_cls=args.single_cls, 
#         mosaic=args.mosaic,
#         optimizer=args.optimizer, 
#         lr0 = args.lr0, 
#         lrf = args.lrf, 
#         momentum=args.momentum
#     )

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/space-dataset/yolo_params.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.0001, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.9, mosaic=0.4, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, 